In [1]:
import boto.ec2
from paramiko import SSHClient
import paramiko
from termcolor import colored
import boto3
import click
import time
from s3_util import *
from ec2 import *
from spot_instance import *

In [2]:
tag = 'smalltest'
new_reservation = request_spot_instances(count=3,tag=tag)
time.sleep(120)
run_command_all_instances(tag=tag,inline=False,target='./init_script.sh')

All have ips assigned.
34.205.174.203 (pending) isn't running.
18.214.25.205 (pending) isn't running.
34.200.214.185 (pending) isn't running.
All aren't running. Wait for 10 sec...
34.205.174.203 is running.
18.214.25.205 (pending) isn't running.
34.200.214.185 (pending) isn't running.
All aren't running. Wait for 10 sec...
34.205.174.203 is running.
18.214.25.205 is running.
34.200.214.185 is running.
All are running.
34.205.174.203
18.214.25.205
34.200.214.185
There're 5 instances.
3 are running now.
--------------------
0 - Executing scripts on 18.214.25.205


NoValidConnectionsError: [Errno None] Unable to connect to port 22 on 18.214.25.205

In [4]:
tag = 'smalltest'
ips=get_all_instances(tag)
ips.sort()
command = "./scripts/runIllinoisTemporal.sh "
partitions=["4 5 6 7 8","9 10 11 12 13","14 15 16 17 18"]
for i,host in enumerate(ips):
    print "--------------------"
    print "%d - Executing scripts on %s" %(i,host)
    run_command(host,command+"\"%s\">>~/log/illinois-temporal/ip-%s.log &" % (partitions[i], host.replace('.','-')), inline=True, verbose=False)

There're 5 instances.
3 are running now.
--------------------
0 - Executing scripts on 18.214.25.205
Run command:./scripts/runIllinoisTemporal.sh "4 5 6 7 8">>~/log/illinois-temporal/ip-18-214-25-205.log &
--------------------
1 - Executing scripts on 34.200.214.185
Run command:./scripts/runIllinoisTemporal.sh "9 10 11 12 13">>~/log/illinois-temporal/ip-34-200-214-185.log &
--------------------
2 - Executing scripts on 34.205.174.203
Run command:./scripts/runIllinoisTemporal.sh "14 15 16 17 18">>~/log/illinois-temporal/ip-34-205-174-203.log &


In [25]:
def partitionsFinished(partitions,cache):
    partitions = partitions.split()
    partitions = [int(x) for x in partitions]
    allFinished = True
    for p in partitions:
        if p in cache:
            continue
        
        if not fileExistsS3('results/illinois-temporal/%d.ser.tgz'%p) or not fileExistsS3('results/illinois-temporal/%d.timeline.tgz'%p):
            print "partition %d unfinished" % p
            allFinished = False
            break
        else:
            print "partition %d finished" %p
            cache.add(p)
    return allFinished

In [8]:
print partitionsFinished("4 5 6")

partition 4 unfinished
False


In [10]:
par2host = {}
for index,ip in enumerate(ips):
    par2host[partitions[index]] = ip
par2host

{'14 15 16 17 18': u'34.205.174.203',
 '4 5 6 7 8': u'18.214.25.205',
 '9 10 11 12 13': u'34.200.214.185'}

In [33]:
def cplog2s3(ip):
    log_name = 'ip-%s.log' % ip.replace('.','-')
    command = "source ~/.customrc; cptos3.sh ~/log/illinois-temporal/%s logs/illinois-temporal/%s" % (log_name,log_name)
    run_command(host,inline=True,verbose=True,command=command)

In [27]:
def stopInstanceByIp(ip):
    conn=boto.ec2.connect_to_region('us-east-1')
    reservations=conn.get_all_instances()
    instances=[i for r in reservations for i in r.instances]
    for ins in instances:
        if ins.ip_address == ip:
            print ins.id+" is stopped now."
            boto.connect_ec2().terminate_instances(ins.id)

In [ ]:
partitions_remaining = partitions
cache = set()
while 1:
    for p_str in partitions_remaining:
        if partitionsFinished(p_str,cache):
            print "%s finished" % p_str
            print "Copy cmd log to s3"
            partitions.remove(p_str)
            host = par2host[p_str]
            cplog2s3(host)
            print "Stop instance"
            stopInstanceByIp(host)
        else:
            print "Wait for 10 mins"
            time.sleep(60*10)
            #time.sleep(5)
    partitions_remaining = partitions

partition 4 finished
partition 5 finished
partition 6 finished
partition 7 finished
partition 8 unfinished
Wait for 10 mins
partition 8 unfinished
Wait for 10 mins
partition 8 unfinished
Wait for 10 mins
partition 8 finished
4 5 6 7 8 finished
Copy cmd log to s3
Run command:source ~/.customrc; cptos3 ~/log/illinois-temporal/ip-18-214-25-205.log logs/illinois-temporal/ip-18-214-25-205.log

bash: cptos3: command not found

Stop instance
i-05f9f765667b84be9 is stopped now.


In [21]:
par2host

{'14 15 16 17 18': u'34.205.174.203',
 '4 5 6 7 8': u'18.214.25.205',
 '9 10 11 12 13': u'34.200.214.185'}